# Supplementary Table 6. DROSHA cleavage patterns
1. Library analyzed
2. Input
3. Cleaved
4. Productively cleaved
5. Normalized cleavage productivity
6. Cleavage site information - chr, 5' CS, 3' CS, strand, relative position from miRBase site, cleavage specificity
7. Cleavage imbalance
8. Cleavage type
9. Build table

In [1]:
import time
today = time.strftime('%Y-%m-%d')
name = 'Seungchan Baek'
print 'Last revised by %s at %s.' % (name, today)

Last revised by Seungchan Baek at 2020-10-13.


In [2]:
home = '/casa/bsc/projects/1_DCS/2004_paper_prep/'
%cd $home

/casa/bsc/projects/1_DCS/2004_paper_prep


In [3]:
from __future__ import division
import sys; sys.path.append('/casa/bsc/notebooks/')
from basic import gen_result_dir
resultpath = gen_result_dir('results/')
print 'resultpath:', resultpath

resultpath: results/201013/


In [4]:
import pandas as pd
import numpy as np
import re
import gzip
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
%matplotlib inline

In [5]:
LIBRARIES = [ 'set1', 'set2', 'set3', 'set4', 'set5' ]

#### Import supplementary tables

In [7]:
s1 = pd.read_csv('supplementary/201012_s1_pri-info.csv', header=1, index_col=0)
s2 = pd.read_csv('supplementary/201012_s2_pri-construct.csv', header=1, index_col=0)
s3 = pd.read_csv('supplementary/201012_s3_input.csv', header=1, index_col=0)
s4 = pd.read_csv('supplementary/201012_s4_cleavage-product.csv', header=1)
print 's1:\t%s'%', '.join(list(s1.columns))
print 's2:\t%s'%', '.join(list(s2.columns)[:-2])
print 's3:\t%s'%', '.join(list(s3.columns))
print 's4:\t%s'%', '.join(list(s4.columns)[:-4])

s1:	5p mature, 5p sequence, 3p mature, 3p sequence, Note
s2:	Chr, Start, End, Strand, Construct sequence, 100way phyloP scores (pre-miRNA -/+ 100nt)
s3:	set1-1, set1-2, set2, set3-1, set3-2, set4, set5-1, set5-2
s4:	Pri-miRNA, rstart, rend, pilot-1, pilot-2, set1-1, set1-2, set2-1, set2-2, set3-1, set3-2


In [8]:
def get_pre_position(pri):
    constructseq = s2.loc[pri, 'Construct sequence'].replace('T','U')
    seq5p = s1.loc[pri, '5p sequence']
    seq3p = s1.loc[pri, '3p sequence']
    if seq5p=='n.a.' or constructseq.find(seq5p)==-1:
        prestart = 0
    else:
        prestart = constructseq.find(seq5p)+1
    if seq3p=='n.a.' or constructseq.find(seq3p)==-1:
        preend = 0
    else:
        preend = constructseq.rfind(seq3p)+len(seq3p)
    return prestart, preend

In [9]:
allpris = s1.index
preposition = { pri:get_pre_position(pri) for pri in allpris }
print len(preposition)

1881


#### Prepare data

In [10]:
inpsum = pd.DataFrame()
for inp in ['set1','set3','set5']:
    inpsum[inp] = s3[['%s-1'%inp,'%s-2'%inp]].sum(axis=1)
inpsum['set2'] = s3['set2']
inpsum['set4'] = s3['set4'] 
inpsum = inpsum[LIBRARIES]
inpsum.head(1)

,set1,set2,set3,set4,set5
Pri-miRNA,,,,,
hsa-let-7a-1,1373,474,0,2,0


In [11]:
clvraw = s4.set_index(['Pri-miRNA','rstart','rend'])
clvtbl = pd.DataFrame()
for lib in LIBRARIES:
    clvtbl[lib] = clvraw[['%s-1'%lib,'%s-2'%lib]].sum(axis=1)
clvtbl = clvtbl.reset_index()
clvtbl.head(1)

,Pri-miRNA,rstart,rend,set1,set2,set3,set4,set5
0,hsa-let-7a-1,1,8,4,4,0,0,0


In [12]:
start, mid, end = 1, 63, 125
clv5f = clvtbl[(clvtbl['rstart']==start)&(clvtbl['rend']<mid)].set_index('Pri-miRNA')
clv3f = clvtbl[(clvtbl['rstart']>mid)&(clvtbl['rend']==end)].set_index('Pri-miRNA')
clvhn = clvtbl[(clvtbl['rstart']<mid)&(clvtbl['rend']>mid)]
clvn5 = clvhn[clvhn['rstart']==start]
clvn3 = clvhn[clvhn['rend']==end]
clvh = clvhn.drop(clvn5.index).drop(clvn3.index).set_index('Pri-miRNA')
clvn5 = clvn5.set_index('Pri-miRNA'); clvn3 = clvn3.set_index('Pri-miRNA')

In [13]:
def fill_unfound(tbl):
    found = set(tbl.index)
    unfound = [ mir for mir in allpris if mir not in found ]
    fill = pd.DataFrame(index=unfound, columns=tbl.columns).fillna(0)
    return tbl.append(fill)

In [14]:
clvtbl = fill_unfound(clvtbl.set_index('Pri-miRNA'))
clv5f = fill_unfound(clv5f)
clv3f = fill_unfound(clv3f)
clvh = fill_unfound(clvh)
clvn5 = fill_unfound(clvn5)
clvn3 = fill_unfound(clvn3)

### 1. Library analyzed

In [15]:
rcutoff = 30

In [16]:
suffinps = list(inpsum[(inpsum>=rcutoff).any(axis=1)].index)
print len(suffinps), suffinps[:3]

1819 ['hsa-let-7a-1', 'hsa-let-7a-2', 'hsa-let-7a-3']


In [17]:
def select_lib(mir, rcutoff):
    return max([lib for lib in LIBRARIES if inpsum.loc[mir,lib]>=rcutoff], 
               key=lambda x: clvtbl.loc[mir,x].sum())

In [18]:
sufflibs = { mir:select_lib(mir, rcutoff) for mir in suffinps }
print sufflibs['hsa-let-7a-1']

set1


### 2. Input count

In [19]:
inpcnts = { mir:inpsum.loc[mir,sufflibs[mir]] for mir in suffinps }
print inpcnts['hsa-let-7a-1']

1373


### 3. Cleavage product counts

In [20]:
def get_clv_all(mir, lib):
    clv5 = clv5f.loc[mir, lib]
    clv3 = clv3f.loc[mir, lib]
    return int(round((clv5.sum()+clv3.sum())/2, 0))

In [21]:
clvcnts = { mir:get_clv_all(mir,sufflibs[mir]) for mir in suffinps }
print clvcnts['hsa-let-7a-1']

16700


### 4. Productively/ Inversely cleaved

In [22]:
window = 3

In [23]:
def get_clv_prod(mir, lib, window):
    ps, pe = preposition[mir]
    if ps==0 and pe==0:
        return -1
    clv5 = clv5f.loc[[mir]].reset_index().set_index('rend')[lib]
    clv3 = clv3f.loc[[mir]].reset_index().set_index('rstart')[lib]
    if ps and pe:
        prod5 = clv5.reindex(range(ps-1-window,ps+window)).sum()
        prod3 = clv3.reindex(range(pe+1-window,pe+2+window)).sum()
        prod = round((prod5+prod3)/2, 0)
    elif ps:
        prod = clv5.reindex(range(ps-1-window,ps+window)).sum()
    else:
        prod = clv3.reindex(range(pe+1-window,pe+2+window)).sum()
    return int(prod)

In [24]:
prodcnts = { mir:get_clv_prod(mir,sufflibs[mir],window) for mir in suffinps }
print prodcnts['hsa-let-7a-1']

16552


In [25]:
def get_clv_inv(mir, lib, window):
    ps, pe = preposition[mir]
    if ps==0 and pe==0:
        return -1
    invs = ps+11
    inve = pe-11
    clv5 = clv5f.loc[[mir]].reset_index().set_index('rend')[lib]
    clv3 = clv3f.loc[[mir]].reset_index().set_index('rstart')[lib]
    if ps and pe:
        prod5 = clv5.reindex(range(invs-1-window,invs+window)).sum()
        prod3 = clv3.reindex(range(inve+1-window,inve+2+window)).sum()
        prod = round((prod5+prod3)/2, 0)
    elif ps:
        prod = clv5.reindex(range(invs-1-window,invs+window)).sum()
    else:
        prod = clv3.reindex(range(inve+1-window,inve+2+window)).sum()
    return int(prod)

In [26]:
invcnts = { mir:get_clv_inv(mir,sufflibs[mir],window) for mir in suffinps }
print invcnts['hsa-let-7a-1']

12


### 5. Normalized cleavage productivity

In [27]:
def get_productivity(mir, lib, window):
    clv = get_clv_all(mir, lib)
    prod = get_clv_prod(mir, lib, window)
    if prod<0: return -999
    if clv==0: return 0
    inp = inpsum[lib][mir]
    return (prod/inp)*(prod/clv)

In [28]:
## norm
norm = 'hsa-mir-6788'
norms = { lib:get_productivity(norm, lib, window) for lib in LIBRARIES }

In [29]:
testset = [ 'hsa-mir-144', 'hsa-let-7a-1', 'hsa-mir-17', 'hsa-mir-200a', 'hsa-mir-15a' ]
for mir in testset:
    prod = get_productivity(mir, sufflibs[mir], window)
    print '%s\t%.3f'%(mir,np.log2(prod/norms[sufflibs[mir]]+1))

hsa-mir-144	4.971
hsa-let-7a-1	5.047
hsa-mir-17	2.610
hsa-mir-200a	2.000
hsa-mir-15a	0.956


In [30]:
normprods = {}
for mir in suffinps:
    lib = select_lib(mir, rcutoff)
    prod = get_productivity(mir, lib, window)
    if prod>-999:
        normprods[mir] = np.log2(prod/norms[lib]+1)

### 6. Cleavage site information - chr, 5' CS, 3' CS, strand, relative position from miRBase site, cleavage specificity

In [32]:
rcutoff = 30
fcutoff = .01

In [33]:
def get_frac_5frag(mir):
    sub = clv5f.loc[[mir]].set_index('rend')
    return (sub/sub.sum())

In [34]:
def get_frac_3frag(mir):
    sub = clv3f.loc[[mir]].set_index('rstart')
    return (sub/sub.sum())

In [35]:
def get_frac_hairpin(mir):
    sub = clvh.loc[[mir]].set_index(['rstart','rend'])
    return (sub/sub.sum())

In [36]:
def filter_frac(frac, fcutoff):
    return frac[frac>=fcutoff]

In [37]:
def add_specificity(row):
    row['cleavage specificity'] = np.log2((row['5frag']+row['hairpin']+row['3frag'])/3+1)
    return row

In [38]:
def add_diff(row):
    if row['pstart'] and row['pend']:
        row['diff'] = min([row['hstart']-row['pstart'], row['pend']-row['hend']], key=abs)
    elif row['pstart']:
        row['diff'] = row['hstart'] - row['pstart'] 
    else:
        row['diff'] = row['pend'] - row['hend']
    return row

In [39]:
def drop_duplicates(sortedtbl):
    if len(sortedtbl)<=1:
        return sortedtbl
    hs = sortedtbl['hstart'].tolist()[0]
    he = sortedtbl['hend'].tolist()[0]
    sub = sortedtbl.iloc[1:]
    sub = sub[(sub['hstart']!=hs)&(sub['hend']!=he)]
    return sortedtbl.iloc[:1].append(drop_duplicates(sub))

In [46]:
frac5f = { mir:get_frac_5frag(mir) for mir in allpris } 
frac3f = { mir:get_frac_3frag(mir) for mir in allpris } 
frachpn = { mir:get_frac_hairpin(mir) for mir in allpris }

In [111]:
COLS = [ 'miRNA', 'pstart', 'pend', 'hstart', 'hend', '5frag', '3frag', 'hairpin', 
         'cleavage specificity', 'diff' ]
def get_dcs(mir, rcutoff, fcutoff, lib):
    ps, pe = preposition[mir]
    clvsites = pd.DataFrame({0:dict(zip(COLS,[mir,ps,pe,1,125,0,0,0,0,99]))}).T[COLS]
    clv5, clv3, clvhpn = [ f[mir][lib] if c.loc[mir,lib].sum()>=rcutoff else pd.Series()
                           for f,c in zip([frac5f,frac3f,frachpn], [clv5f,clv3f,clvh]) ]
    if sum(map(bool, [clv5.to_dict(),clv3.to_dict(),clvhpn.to_dict()]))<2:
        return clvsites
    clv52 = defaultdict(float); clv52.update(clv5)
    clv32 = defaultdict(float); clv32.update(clv3)
    clvhpn2 = defaultdict(float); clvhpn2.update(clvhpn)
    for c5, frac5 in filter_frac(clv5, fcutoff).items():
        for c3, frac3 in filter_frac(clv3, fcutoff).items():
            row = dict(zip(COLS,[mir,ps,pe,c5+1,c3-1,frac5,frac3,clvhpn2[(c5+1,c3-1)]]))
            clvsites = clvsites.append(row, ignore_index=True)
    for (c5,c3), frach in filter_frac(clvhpn, fcutoff).items():
        row = dict(zip(COLS,[mir,ps,pe,c5,c3,clv52[c5-1],clv32[c3+1],frach]))
        clvsites = clvsites.append(row, ignore_index=True)
    cs = clvsites.apply(add_specificity,axis=1).sort_values('cleavage specificity').iloc[::-1]
    return drop_duplicates(cs).apply(add_diff,axis=1)

In [143]:
get_dcs('hsa-let-7a-1', rcutoff, fcutoff, 'set1').head()

,miRNA,pstart,pend,hstart,hend,5frag,3frag,hairpin,cleavage specificity,diff
7,hsa-let-7a-1,25,96,25,96,0.938918,0.975172,0.733558,0.912688,0
2,hsa-let-7a-1,25,96,24,97,0.039322,0.012130,0.001686,0.025331,-1
9,hsa-let-7a-1,25,96,42,80,0.000363,0.000213,0.045531,0.022004,16
10,hsa-let-7a-1,25,96,43,79,0.000000,0.000213,0.011804,0.005768,17
0,hsa-let-7a-1,25,96,1,125,0.000000,0.000000,0.000000,0.000000,-24


In [112]:
print time.ctime()
clvall = pd.DataFrame(columns=COLS)
for mir in suffinps:
    lib = select_lib(mir, rcutoff)
    clvall = clvall.append(get_dcs(mir, rcutoff, fcutoff, lib), ignore_index=True)
clvall = clvall[COLS]
print time.ctime()
print '# of all sites:\t%s' % len(clvall)
print '# of miRNAs:\t%s' % len(set(clvall['miRNA']))

Tue Jun 23 15:13:34 2020
Tue Jun 23 15:35:44 2020
# of all sites:	23341
# of miRNAs:	1819


In [31]:
#clvall.to_csv('results/200623/200623_cleavage_specificity_MPonly.csv')
clvall = pd.read_csv('results/200623/200623_cleavage_specificity_MPonly.csv', index_col=0)

In [32]:
def get_clv_info(mir, hstart, hend):
    chrom, pstart, pend, strand = s2.loc[mir,['Chr','Start','End','Strand']]
    if strand=='+':
        clv5 = pstart+hstart-1
        clv3 = pstart+hend-1
    else:
        clv5 = pend-hstart+1
        clv3 = pend-hend+1
    return chrom, clv5, clv3, strand

In [33]:
print get_clv_info('hsa-let-7a-1', 25, 96)

('chr9', 94175962, 94176033, '+')


### 7. Cleavage imbalance

In [34]:
print (clvall['5frag']-clvall['3frag']).head()

0   -0.036254
1    0.027192
2    0.000150
3   -0.000213
4    0.000000
dtype: float64


### 8. Cleavage type

In [35]:
window = 3
offset = 11
procut = 1.4
specut = .42
altcut = .26
imbcut = .2

In [36]:
overlap = [ mir for mir in set(clvall['miRNA']) if mir in normprods ]
print len(overlap)

1816


In [37]:
def get_dcs_all(procut, specut, window):
    clvpro = clvall[abs(clvall['diff'])<=window]
    clvsig = clvpro[clvpro['cleavage specificity']>=specut]
    return [ mir for mir in set(clvsig['miRNA']) if normprods[mir]>=procut ]

In [38]:
def get_dcs_multi(procut, specut, altcut, window):
    clvpro = clvall[abs(clvall['diff'])<=window]
    clvsig = clvpro[clvpro['cleavage specificity']>=specut]
    clvcnts = Counter(clvpro[clvpro['cleavage specificity']>=altcut]['miRNA'])
    return [ mir for mir in set(clvsig['miRNA']) if normprods[mir]>=procut and clvcnts[mir]>1 ]

In [39]:
def get_dcs_single(procut, specut, altcut, window):
    dcsmirs = get_dcs_all(procut, specut, window)
    multi = get_dcs_multi(procut, specut, altcut, window)
    return [ mir for mir in dcsmirs if mir not in multi ]

In [40]:
def get_invert(specut, offset, window, mirexc):
    clvinv = clvall[abs(clvall['diff']-offset)<=window]
    clvsig = clvinv[clvinv['cleavage specificity']>=specut]
    return [ mir for mir in set(clvsig['miRNA']) if mir not in mirexc ]

In [41]:
def get_nick5(imbcut, window, mirexc):
    clvpro = clvall[(abs(clvall['diff'])<=window)&(clvall['3frag']>0)]
    clvn5 = clvpro[(clvpro['5frag']-clvpro['3frag'])>=imbcut]
    return [ mir for mir in set(clvn5['miRNA']) if mir not in mirexc ]

In [42]:
def get_nick3(imbcut, window, mirexc):
    clvpro = clvall[(abs(clvall['diff'])<=window)&(clvall['5frag']>0)]
    clvn3 = clvpro[(clvpro['3frag']-clvpro['5frag'])>=imbcut]
    return [ mir for mir in set(clvn3['miRNA']) if mir not in mirexc ]

In [43]:
dcsmirs = get_dcs_all(procut, specut, window)
single = get_dcs_single(procut, specut, altcut, window)
multi = get_dcs_multi(procut, specut, altcut, window)

nick5 = get_nick5(imbcut, window, dcsmirs)
nick3 = get_nick3(imbcut, window, dcsmirs+nick5)
nick = list(set(nick5+nick3))

inverted = get_invert(specut, offset, window, dcsmirs+nick)
nodcs = [ mir for mir in overlap if mir not in dcsmirs ]
nonspec = [ mir for mir in nodcs if mir not in inverted+nick ]

print 'Productive: %s, Unproductive: %s' % (len(dcsmirs), len(nodcs))
print 'Single: %s, Multiple: %s, Inverted: %s, Nick: %s, Non-specific: %s'\
% (len(single), len(multi), len(inverted), len(nick), len(nonspec))
print "5' nick: %s, 3' nick: %s" % (len(nick5), len(nick3))

Productive: 512, Unproductive: 1304
Single: 445, Multiple: 67, Inverted: 156, Nick: 107, Non-specific: 1041
5' nick: 72, 3' nick: 35


In [44]:
print 'let-7a-1 in unique:', 'hsa-let-7a-1' in single
print 'mir-342 in multiple:', 'hsa-mir-342' in multi
print 'mir-17 in nick5:', 'hsa-mir-17' in nick5
print 'mir-15a in nick5:', 'hsa-mir-15a' in nick5

let-7a-1 in unique: True
mir-342 in multiple: True
mir-17 in nick5: True
mir-15a in nick5: True


In [45]:
clvtypes = {}
clvtypes.update({mir:'single' for mir in single})
clvtypes.update({mir:'multiple' for mir in multi})
clvtypes.update({mir:'inverted' for mir in inverted})
clvtypes.update({mir:'nick5' for mir in nick5})
clvtypes.update({mir:'nick3' for mir in nick3})
clvtypes.update({mir:'non-specific' for mir in nonspec})
print len(clvtypes)

1816


#### 9. Build table

In [46]:
cols = [ 'Pri-miRNA', 'Library analyzed', 'Input', 'Cleaved', 'Productively cleaved',
         'Inversely cleaved','Cleavage Productivity', 'Chr', "5' cleavage site", 
         "3' cleavage site",'Strand', 'Relative position from miRBase site', 
         'Cleavage Specificity', 'Cleavage Imbalance', 'Cleavage type' ]

In [47]:
clvpro = clvall[(clvall['diff'].apply(abs)<=window)&(clvall['cleavage specificity']>0)]
print len(set(clvpro['miRNA']))

1648


In [48]:
tbl = pd.DataFrame(columns=cols)
for i, row in clvpro.iterrows():
    mir, hs, he, cs, diff = row[['miRNA','hstart','hend','cleavage specificity','diff']]
    chrom, cs5, cs3, strand = get_clv_info(mir, hs, he)
    imbal = row['5frag'] - row['3frag']
    ctype = clvtypes[mir]
    tbl = tbl.append(dict(zip(cols,[mir, sufflibs[mir],inpcnts[mir],clvcnts[mir],prodcnts[mir],
                                    invcnts[mir],normprods[mir],chrom,cs5,cs3,strand,diff, 
                                    cs,imbal,ctype])), ignore_index=True)

In [49]:
nonpro = [ mir for mir in overlap if mir not in set(clvpro['miRNA']) ]
for mir in nonpro:
    tbl = tbl.append(dict(zip(cols,[mir,sufflibs[mir],inpcnts[mir],clvcnts[mir],prodcnts[mir], 
                                    invcnts[mir],normprods[mir],'n.a.',0,0,'n.a.',0,0,0, 
                                    clvtypes[mir]])), ignore_index=True)

In [50]:
tbl = tbl.sort_values('Pri-miRNA').set_index('Pri-miRNA')
print len(set(tbl.index))

1816


In [51]:
tbl.to_csv('resources/201012_s6_cleavage-patterns.csv')

In [52]:
out = open('supplementary/201012_s6_cleavage-patterns.csv', 'wt')
description = 'Supplementary Table 6. Cleavage patterns\n\n\n\n\n'
out.write(description)
for l in open('resources/201012_s6_cleavage-patterns.csv', 'rt'):
    out.write(l)
out.close()